Need these libraries:

In [6]:
# install.packages("caret")
# install.packages("BBmisc")
# install.packages("data.table")

library(caret)
library(BBmisc)
library(data.table)

We'll use the iris dataset for exemplifying:

In [7]:
data(iris)
head(iris)
dim(iris)
summary(iris)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


[1] 150   5

  Sepal.Length    Sepal.Width     Petal.Length    Petal.Width   
 Min.   :4.300   Min.   :2.000   Min.   :1.000   Min.   :0.100  
 1st Qu.:5.100   1st Qu.:2.800   1st Qu.:1.600   1st Qu.:0.300  
 Median :5.800   Median :3.000   Median :4.350   Median :1.300  
 Mean   :5.843   Mean   :3.057   Mean   :3.758   Mean   :1.199  
 3rd Qu.:6.400   3rd Qu.:3.300   3rd Qu.:5.100   3rd Qu.:1.800  
 Max.   :7.900   Max.   :4.400   Max.   :6.900   Max.   :2.500  
       Species  
 setosa    :50  
 versicolor:50  
 virginica :50  
                
                
                

In [10]:
## Method 1 for MSE 
k<- 10
train<- sample(1:nrow(iris), size=nrow(iris)*(k-1)/k, replace=FALSE)
	df_train <- iris[train, ]
	df_test <- iris[-train, ]
	fit <- lm(df_train[,1]~., data=df_train)

fit_summ <- summary(fit)
mean(fit_summ$residuals^2)

Warning message in summary.lm(fit):
"essentially perfect fit: summary may be unreliable"

[1] 8.447777e-31

In [11]:
## Method 2 for MSE
data <- data.frame(pred=predict(fit), actual= df_train[,1])
head(data)
mean((data$actual - data$pred)^2)

,pred,actual
8,5.0,5.0
42,4.5,4.5
68,5.8,5.8
19,5.7,5.7
115,5.8,5.8
69,6.2,6.2


[1] 1.706277e-30

The second method is generally more reliable. I'm not entirely sure why. We'll use that any way.

We'll initialize the MSE computation on the first feature of the iris dataset -- this is essentially what we need to really do for CV! We'll explore it on the other features afterwards.

In [12]:
k<- 10
m <- vector(length=k-1)

while(k>1) {
	train<- sample(1:nrow(iris), size=nrow(iris)*(k-1)/k, replace=FALSE)
	df_train <- iris[train, ]
	df_test <- iris[-train, ]
	iris <- iris[train, ]
	k <- k-1
	fit <- lm(df_train[,1]~., data=df_train)
	data <- data.frame(pred=predict(fit), actual= df_train[,1])
	m[k] <- mean((data$actual - data$pred)^2)
	}


In [13]:
loss <- mean(m)
loss

[1] 3.927655e-31

Extend it to **all features** instead:

In [16]:
options(warn=-1)
K<- 10
k<- 10
m <- vector(length=(k-1)*ncol(iris))
j<- 1

for(i in 1:ncol(iris)) {
	# j<- j+ncol(iris)
	k<- 10
	data(iris)
	while(k>1) {
		set.seed(10)
		train<- sample(1:nrow(iris), size=nrow(iris)*(k-1)/k, replace=FALSE)
		df_train <- iris[train, ]
		df_test <- iris[-train, ]
		iris <- iris[train, ]
		k <- k-1
		fit <- lm(df_train[,i]~., data=df_train)
		data <- data.frame(pred=predict(fit), actual= df_train[,1])
		m[j] <- mean((data$actual - data$pred)^2)
		j<- j+1
		}
	}


In [17]:
options(warn=0) # The warnings come from the feature of iris which is characters
                ## Obviously needs addressing on what we end up working on!

Let's chunk our results accordingly. At the moment, the resulting vector holds the losses throughout all datasets AND throghout all features. Fortunately, they're well ordered so we can separate them:

In [19]:
chunk_length <- K-1
m1 <- split(m, ceiling(seq_along(m) / chunk_length))
m1<- as.vector(m1)

## Alternatively:
	# library(BBmisc)
	# chunked <- chunk(m, chunk.size=K-1)

In [20]:
m1 <- as.data.frame(m1)
m1 <- transpose(transpose(m1))

In [21]:
head(m1)

V1,V2,V3,V4,V5
0.000000e+00,8.850593,5.368370,21.75356,14.95570
1.932709e-30,8.687250,5.534167,21.80692,14.86567
4.282388e-31,8.790190,5.546095,21.83667,14.86790
7.538004e-31,8.789778,5.780111,22.20567,15.24733
6.731613e-31,9.067467,5.515733,22.13520,15.24520
2.235106e-31,9.496500,5.184167,22.16033,15.27017


That's all losses throughout all features and on all (9) testing partitions attempted. They're all sampled WITHOUT replacement, so we're certain to get a hold of the best out of them this way!

In [23]:
loss <- vector(len=ncol(m1))
for(i in 1:ncol(m1)){
	loss[i] <- mean(m1[, i])
	}

for(i in 1:length(loss)) {
	print(paste("The MSE of the fitted column", i, "is", loss[i], sep=" "))
	}

[1] "The MSE of the fitted column 1 is 7.98266019364388e-31"
[1] "The MSE of the fitted column 2 is 9.08334564961787"
[1] "The MSE of the fitted column 3 is 5.38576284538507"
[1] "The MSE of the fitted column 4 is 21.8025808641975"
[1] "The MSE of the fitted column 5 is 14.9474416813639"


Iris was originally constructed to model the first column by all others, much like the 'cars' dataset and others. It's expected the first feature will fit so well (also why our first method of lm'ing gave a nearly perfect fit!).

In [24]:
overall_loss <- mean(loss, na.rm=TRUE)
overall_loss

[1] 10.24383

This provides a relevant measurement of how successful our CV was.